# Генератор обучающей выборки (песок)

Равномерное заполнение изображения 100x100 px "песчинками" - кругами разного радиуса

In [1]:
from PIL import Image, ImageDraw
from numpy.random import randint, randn
from numpy import pi
from tqdm import tqdm

### Гиперпараметры

In [2]:
# коэффициент линейного тренда r = Kx
K = 0.01
# размеры изображения обучающей выборки
H = 100
W = 100
# ширина панорамы
Wpan = 500
# граничная плотность заполнения изображения "песком" (R = S/S0)
R0 = 0.95

### Функции генерирования сэмплов

Изображения side1, side2 получаются путем отрезания кусков 100x100 от панорамы

In [3]:
def genSamplePanorama(W, Wpan, H, K, R0, i, val=False):
    # новое ч/б изображение размером в 10 раз больше для суперсэмплинга
    imagePan = Image.new('1', (10 * Wpan, 10 * H), 'white')
    draw = ImageDraw.Draw(imagePan)
    # S - текущая занятая площадь, S0 - общая площадь "холста"
    S = 0
    R = 0
    S0 = 100 * Wpan * H
    # рисуем "песчинки", пока не достигнем плотности R0
    while(R < R0):
        x0 = randint(1, 10 * Wpan)
        y0 = randint(1, 10 * H)
        r = abs(K * x0 + int(randn()))
        S += pi * r**2
        R = S / S0
        draw.ellipse((x0 - r, y0 - r, x0 + r, y0 + r), fill='black')
    del draw
    # суперсэмплинг с антиалиасингом
    imagePan = imagePan.resize((Wpan, H), Image.LANCZOS)
    fileName = "sample" + str(i) + ".jpg"
    filePath = "../data/sand/trend1/"
    if (val == False):
        filePath += "train/"
    else:
        filePath += "validation/"
    imagePan.save(filePath + "panorama/" + fileName)
    
    imageSide1 = imagePan.crop((0, 0, W, H))
    imageSide1.save(filePath + "side1/" + fileName)
    
    imageSide2 = imagePan.crop((Wpan - W, 0, Wpan, H))
    imageSide2.save(filePath + "side2/" + fileName)

In [5]:
# количество сэмплов для генерации
N = 1000
# train/validation = 80/20
N_train = int(0.8 * N)
N_val = N - N_train
print('Generating train dataset:')
for i in tqdm(range(N_train)):
    genSamplePanorama(W, Wpan, H, K, R0, i)

print('Generating validation dataset:')
for i in tqdm(range(N_val)):
    genSamplePanorama(W, Wpan, H, K, R0, i, val=True)

  0%|          | 1/800 [00:00<02:36,  5.11it/s]

Generating train dataset:


  0%|          | 1/200 [00:00<00:37,  5.25it/s]

Generating validation dataset:


100%|██████████| 200/200 [00:34<00:00,  5.93it/s]
